# Getting started for the AML 2023/2024 Egocentric Vision Project

<a href="https://colab.research.google.com/github/EgovisionPolito/aml23-ego/blob/master/colab_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the code

In [7]:
# Installing missing dependencies
# !pip install omegaconf coloredlogs wandb

## EPIC-Kitchens-55 dataset

**READ carefully!**

To develop the project, you need to download the RGB frames for a subset of EPIC-Kitchens-55 (participants P08, P01 and P22) from [here](https://drive.google.com/drive/u/1/folders/1dJOtZ07WovP3YSCRAnU0E4gsfqDzpMVo). 

You also need to the pretrained checkpoints for each domain from [here](https://politoit-my.sharepoint.com/:f:/g/personal/simone_peirone_polito_it/ErdsZhvmR65Lun5_5O0-l5sBTPjCCZZq2f700Tj_CNzjTQ?e=L1yflf).

Add the Google Drive directory containing the dataset to your Google Drive or upload the dataset on your Google Drive to access it from Google Colab.

**NOTE**: As the dataset is quite heavy, we stronly suggest you to implement and test all your code on one for the three dataset. Then, once you are sure everything works, repeat the experiments on the remaining two datasets.

## Features extraction

In [10]:
%%bash

python.exe save_feat.py name=uniform_5 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  split=train \
  dataset.workers=0 \
  save.num_frames_per_clip.RGB=5 \
  save.dense_sampling.RGB=False

# python.exe save_feat.py name=uniform_25 config=configs/I3D_save_feat.yaml dataset.shift=D1-D1 dataset.RGB.data_path=../ek_data/frames split=test dataset.workers=0 save.num_frames_per_clip.RGB=25 save.dense_sampling.RGB=False
# If everything is working, you should expect an error message telling you to implement the '_get_val_indices' method in the dataset class.
# Once you have implemented it, you should run the script for the train and test split of the dataset to extract the features.

2024-02-17 16:17:29 LOG INFO Feature Extraction
2024-02-17 16:17:29 LOG INFO Running with parameters: 
  action: save
  name: dense_5
  modality: ['RGB']
  total_batch: 128
  batch_size: 32
  gpus: None
  wandb_name: None
  resume_from: ./saved_models/I3D_SourceOnlyD1
  logname: save_D1-D1.log
  models_dir: saved_models\dense_5\Feb17_16-17-23
  train:
    num_iter: 5000
    lr_steps: 3000
    eval_freq: 50
    num_clips: 1
    dense_sampling:
      RGB: True
    num_frames_per_clip:
      RGB: 16
  test:
    num_clips: 5
    dense_sampling:
      RGB: True
    num_frames_per_clip:
      RGB: 16
  dataset:
    annotations_path: train_val
    shift: D1-D1
    workers: 0
    stride: 2
    resolution: 224
    RGB:
      data_path: ../ek_data/frames
      tmpl: img_{:010d}.jpg
      features_name: test_feat_kinetics
    Event:
      rgb4e: 6
  models:
    RGB:
      model: I3D
      normalize: False
      kwargs:
      lr_steps: 3000
      lr: 0.01
      sgd_momentum: 0.9
      weight_decay

In [16]:
%%bash

python.exe plot_features.py \
    use_frames=False \
    features_path='saved_features/uniform_5_D1_test.pkl' \
    split_path='train_val/D1_test.pkl' \
    output_image_path='plots/uniform/uniform_5_test_woim.png' \
    plot_3D=False

{'use_frames': False, 'features_path': 'saved_features/uniform_5_D1_test.pkl', 'split_path': 'train_val/D1_test.pkl', 'output_image_path': 'plots/uniform/uniform_5_test_woim.png', 'plot_3D': False}


B:\Programmi\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
B:\Programmi\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Figure(640x480)


In [1]:
%%bash

python.exe train_classifier.py \
    name=test_train \
    dataset.shift=D1-D1 \
    dataset.RGB.data_path=../ek_data/frames \
    dataset.RGB.features_name=SAVE_DENSE-extracted \
    dataset.workers=0 \
    # config=configs/I3D_save_feat.yaml \
    # action=train \
    # split=train \

In [16]:
import pandas as pd

data = pd.DataFrame(pd.read_pickle('./emg/S00_2.pkl'))
l = list(data['myo_right_readings'][1:])
first = l[0]
first = abs(first)
first = [2*(el-min(el))/(max(el)-min(el))-1 for el in first]
# print(first)

print(data.columns)
freq = 5
row = 3
l = list(data['myo_right_timestamps'][1:])

indexes = []
for i in range(len(l[row])-1):
    if cum > 1/freq or len(indexes) == 0:
        indexes.append(i)
        cum = 0
    else:
        cum += l[row][i+1]-l[row][i]

print(indexes, len(indexes))

Index(['description', 'start', 'stop', 'myo_left_timestamps',
       'myo_left_readings', 'myo_right_timestamps', 'myo_right_readings'],
      dtype='object')
[0, 29, 63, 94, 125, 155, 185, 224, 261, 292, 327, 366, 405, 448, 489, 528, 561, 591, 625, 655, 686, 717, 749, 782, 818, 851, 885, 918, 949, 982, 1015, 1047, 1081, 1115, 1148, 1176, 1203, 1238, 1275, 1316, 1355, 1396, 1435, 1474, 1517, 1556, 1585, 1620, 1655, 1686, 1717, 1751, 1782, 1814, 1845, 1873, 1906, 1937, 1969, 2002, 2036, 2068, 2101, 2134, 2163, 2201, 2238, 2270, 2301, 2337, 2379, 2415, 2450, 2489, 2525, 2554, 2584, 2614, 2646, 2677, 2708, 2739, 2774, 2807, 2839, 2870, 2903, 2934, 2970, 3004, 3032, 3063, 3097, 3128, 3162, 3199, 3240, 3275, 3314, 3352, 3391, 3432, 3470, 3507, 3538, 3571, 3603, 3632, 3668, 3697, 3730, 3763, 3796, 3831, 3862, 3897, 3929, 3962, 3994, 4024, 4056, 4086, 4120, 4158, 4199, 4240, 4275, 4315, 4354, 4393, 4431, 4473, 4512, 4542, 4575, 4609, 4641, 4673, 4707, 4739, 4767, 4797, 4830, 4859, 4890, 4918,

In [62]:
import pandas as pd
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

data = pd.DataFrame(pd.read_pickle('./emg/S00_2.pkl'))

fc = 5
order = 4
to_read = ['myo_right', 'myo_left']

for side in to_read:
    for i, _ in data.iterrows():
        if i != 0:
            # Rectify channels
            row = abs(data.loc[i, side + '_readings'])
            times = data.loc[i, side + '_timestamps']
            
            # Low-pass filter 5 Hz
            fs = times.size / (times[-1]-times[0])

            nyq = 0.5 * fs
            wn = fc / nyq
            
            b, a = butter(order, wn, 'low', analog=False)
            filtered = filtfilt(b, a, row.T).T
            
            # Normalize in [-1, 1]
            scaler = MinMaxScaler(feature_range=(-1, 1))
            filtered = scaler.fit_transform(filtered)
            
            # if row.size - filtered.size != 0:
            #     print(row.size-filtered.size)
            # if i == 1:
            #     plt.plot(times, row[:, 0], label='Originali')
            #     plt.plot(times, filtered[:, 0], label='Filtered')
            #     plt.grid(True)
            #     plt.show()